# snowflakeR Quickstart -- Local Environment

This notebook is for **local R environments** (RStudio, Posit Workbench, JupyterLab with R kernel).
For Snowflake Workspace Notebooks, use `workspace_quickstart.ipynb`.

**Sections:**
1. Setup
2. Connect to Snowflake
3. Queries & Table Operations
4. DBI & dbplyr Integration
5. Visualization with ggplot2
6. Cleanup

## 1. Setup

Install snowflakeR and set up the Python environment (one time only).

```r
# install.packages("pak")
# pak::pak("Snowflake-Labs/snowflakeR")
# snowflakeR::sfr_install_python_deps()
```

In [ ]:
library(snowflakeR)

---
## 2. Connect to Snowflake

Locally, `sfr_connect()` reads `~/.snowflake/connections.toml` or accepts explicit parameters.

In [ ]:
# Default connection from connections.toml
conn <- sfr_connect()
conn

In [ ]:
# Or use explicit parameters:
# conn <- sfr_connect(
#   account       = "xy12345.us-east-1",
#   user          = "MYUSER",
#   warehouse     = "COMPUTE_WH",
#   database      = "MY_DB",
#   schema        = "PUBLIC",
#   authenticator = "externalbrowser"
# )

In [ ]:
# Set warehouse, database, and schema if not already set
conn <- sfr_use(conn, warehouse = "COMPUTE_WH", database = "MY_DB", schema = "PUBLIC")
sfr_status(conn)

---
## 3. Queries & Table Operations

In [ ]:
# Run a SQL query
result <- sfr_query(conn, "SELECT CURRENT_TIMESTAMP() AS now, CURRENT_USER() AS user_name")
result

In [ ]:
# Write a data.frame to Snowflake
sfr_write_table(conn, "SFR_MTCARS", mtcars, overwrite = TRUE)

In [ ]:
# List tables
tables <- sfr_list_tables(conn)
head(tables, 10)

In [ ]:
# Read it back
df <- sfr_read_table(conn, "SFR_MTCARS")
head(df, 5)

In [ ]:
# Describe columns
sfr_list_fields(conn, "SFR_MTCARS")

---
## 4. DBI & dbplyr Integration

In [ ]:
library(DBI)

DBI::dbGetQuery(conn, "SELECT 42 AS answer")
DBI::dbExistsTable(conn, "SFR_MTCARS")

In [ ]:
library(dplyr)
library(dbplyr)

# Lazy reference to Snowflake table
cars_tbl <- tbl(conn, "SFR_MTCARS")

# dplyr pipeline -- generates SQL, runs on collect()
summary <- cars_tbl |>
  group_by(cyl) |>
  summarise(
    n       = n(),
    avg_mpg = mean(mpg, na.rm = TRUE),
    avg_hp  = mean(hp, na.rm = TRUE)
  ) |>
  arrange(cyl) |>
  collect()

summary

---
## 5. Visualization with ggplot2

In [ ]:
library(ggplot2)

df <- sfr_read_table(conn, "SFR_MTCARS")

ggplot(df, aes(x = wt, y = mpg, color = factor(cyl))) +
  geom_point(size = 3) +
  labs(title = "MPG vs Weight by Cylinder Count",
       x = "Weight (1000 lbs)", y = "Miles per Gallon",
       color = "Cylinders") +
  theme_minimal()

---
## 6. Cleanup

In [ ]:
sfr_execute(conn, "DROP TABLE IF EXISTS SFR_MTCARS")
sfr_execute(conn, "DROP TABLE IF EXISTS SFR_QUICKSTART_TEST")
cat("Cleanup complete.\n")